In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Tissue - oncogene correaltion (failed test)

In [2]:
compacted = np.loadtxt(open("truncated_compacted.csv", "rb"), delimiter=",", skiprows=1)
compacted = np.array(compacted, dtype="int")

In [3]:
def compute_avg_hamming_dist(mtx):
    avg_dist = 0
    for i in range(len(mtx) - 1):
        for j in range(i, len(mtx)):
            hamming_dist = 0
            a = (mtx[i] == 0)
            b = (mtx[j] == 0)
            for k in range(len(a)):
                #if mtx[i][k] != mtx[j][k]:
                if a[k] != b[k]:
                    hamming_dist += 1
            avg_dist += hamming_dist/len(a)
    return avg_dist

In [4]:
def random_tissue_idx(mtx):
    result = np.array(mtx)
    i = 0
    for row in result:
        np.random.shuffle(row)
    return result

In [5]:
compute_avg_hamming_dist(compacted)

70.00000000000017

In [6]:
new = random_tissue_idx(compacted)
compute_avg_hamming_dist(new)

102.11111111111133

### The test above only shows that some of the oncogenes are tissue-specified but does not give any clue on which oncogene is tissue-specified and which is not

# Permutation test